### 安装依赖包 

```sh
uv add alibabacloud_alidns20150109 alibabacloud_credentials alibabacloud_tea_openapi alibabacloud_tea_util alibabacloud_tea_console
```

初始化配置对象 ```alibabacloud_tea_openapi.Config``` 。
```Config``` 对象存放存放 ```access_key_id``` 、```access_key_secret``` 和 ```endpoint``` 等配置，Endpoint 请参考 https://api.aliyun.com/product/Alidns

In [8]:
import os
import dotenv
from alibabacloud_tea_openapi import models as open_api_models

dotenv.load_dotenv()

access_key_id = os.getenv('ACCESS_KEY_ID')
access_key_secret = os.getenv('ACCESS_KEY_SECRET')

if access_key_id is None or access_key_secret is None:
    raise ValueError("ACCESS_KEY_ID and ACCESS_KEY_SECRET must be set in environment variables.")


config = open_api_models.Config(
    access_key_id=access_key_id,
    access_key_secret=access_key_secret,
    endpoint='dns.aliyuncs.com'
)

实例化一个客户端，从 ```alibabacloud_alidns20150109.Client``` 类生成对象 ```client``` 。 
后续 ```request```、```response``` 从 ```alibabacloud_alidns20150109.models``` 中获得。

In [9]:
from alibabacloud_alidns20150109.client import Client

client = Client(config)

创建对应 API 的 ```Request``` 。 方法的命名规则为 API 名称加上 ```Request``` 。

In [10]:
from alibabacloud_alidns20150109 import models

request = models.DescribeDomainRecordsRequest()

设置请求类 ```request``` 的参数。 通过设置 ```request``` 类的属性设置参数，即 API 中必须要提供的信息。例如：

In [11]:
request.lang = 'zh'
request.domain_name = '3ncounter.cn'

通过 client 对象获得对应 ```request``` 响应 response 。

In [12]:
response = client.describe_domain_records(request)
print(response.body)

{'DomainRecords': {'Record': [{'CreateTimestamp': 1750923857189, 'DomainName': '3ncounter.cn', 'Line': 'default', 'Locked': False, 'RR': 'ddns', 'RecordId': '1938141249295366144', 'Status': 'ENABLE', 'TTL': 600, 'Type': 'AAAA', 'UpdateTimestamp': 1750923857412, 'Value': '240e:331:208e:b900:820f:42a2:d3f7:a30b', 'Weight': 1}, {'CreateTimestamp': 1717405995000, 'DomainName': '3ncounter.cn', 'Line': 'default', 'Locked': False, 'RR': 'github', 'RecordId': '898778572241994752', 'Status': 'ENABLE', 'TTL': 600, 'Type': 'CNAME', 'UpdateTimestamp': 1717405995000, 'Value': 'oct-17.github.io', 'Weight': 1}]}, 'PageNumber': 1, 'PageSize': 20, 'RequestId': '2FCE5545-A1CB-5882-B39D-C5788621F56E', 'TotalCount': 2}


In [18]:
def getDomainRecords():
    # 解析记录列表
    request = models.DescribeDomainRecordsRequest()
    request.lang = 'zh'
    request.domain_name = '3ncounter.cn'
    response = client.describe_domain_records(request)
    return response

def deleteDomainRecords(record_id):
    # 删除解析记录
    request = models.DeleteDomainRecordRequest()
    request.record_id = record_id
    response = client.delete_domain_record(request)
    return response

def updateDomainRecord(record_id, rr, value):
    # 更新解析记录
    request = models.UpdateDomainRecordRequest()
    request.record_id = record_id
    request.rr = rr
    request.value = value
    request.type = 'AAAA'
    response = client.update_domain_record(request)
    return response

In [ ]:
import requests

ipv6 = 'http://ipv6.lookup.test-ipv6.com/ip/'
ipv4 = 'http://ipv4.lookup.test-ipv6.com/ip/'

def getIpv6():
    ipv6res = requests.get(ipv6)
    return ipv6res.json()['ip']

def getIpv4():
    ipv4res = requests.get(ipv4)
    return ipv4res.json()['ip']

print(getIpv4())
print(getIpv6())

46.232.121.49
2407:cdc0:b010::49


In [19]:
# import requests

# res = requests.get('http://ip-api.com/json')
# print(res.json())

# 获取域名解析记录列表
res = getDomainRecords()

aaaaRecord = None

for item in res.body.domain_records.record:
    if item.type == 'AAAA':
        aaaaRecord = item
        break

if aaaaRecord is not None:
    ipv6 = getIpv6()
    ret = updateDomainRecord(aaaaRecord.record_id, aaaaRecord.rr, ipv6)
    print(ret)

{'headers': {'date': 'Thu, 26 Jun 2025 09:10:24 GMT', 'content-type': 'application/json;charset=utf-8', 'content-length': '85', 'connection': 'keep-alive', 'keep-alive': 'timeout=25', 'access-control-allow-origin': '*', 'access-control-expose-headers': '*', 'x-acs-request-id': 'A1B8D9F6-DB6B-5CE9-89AD-1227AA8E53CF', 'x-acs-trace-id': '6e89b3d6130ef2e5ca87120c897b1c31', 'etag': '8mXOuhqhACr6AgW1HnG8ksw5'}, 'statusCode': 200, 'body': {'RecordId': '1938141249295366144', 'RequestId': 'A1B8D9F6-DB6B-5CE9-89AD-1227AA8E53CF'}}


In [16]:
# 删除指定的解析记录
# res = deleteDomainRecords('895164709898411008')

使用 try...except... 处理报错。

In [17]:

# from Tea.exceptions import UnretryableException

# try:
#     response = client.add_custom_line(request)
#     print(response)
# except UnretryableException as e:
#     print(e)
# except Exception as e:
#     print(e)